# Description:
To be done

### Imports

In [1]:
#import argparse
#from array import array
import glob
import math
import numpy as np
import os
import ROOT
from ROOT import TCanvas, TFile, TProfile, TNtuple, TH1F, TH2F, TH3F, TH1, TLine
#from ROOT import gROOT, gBenchmark, gRandom, gSystem, Double, gStyle
import scipy
from scipy import stats
import time

Welcome to JupyROOT 6.22/02


### ROOT Batch Mode

In [2]:
# ROOT.gROOT.SetBatch(1)

### Paths

In [3]:
# ============================================================
# Set paths
# ============================================================
datapath = '/data/SingleModule_Nov2020/LArPix/dataRuns/rootTrees/combined_with_light'
print(' datapath:   ', datapath)

outputpath = '/home/lhep/PACMAN/larpix-analysis/lightCharge_anticorrelation'
print(' outputpath: ', outputpath)

files = sorted([os.path.basename(path) for path in glob.glob(datapath+'/*.root')])
print(' datafiles (', len(files), '): ')
for f in files:
    print('              ', f)

 datapath:    /data/SingleModule_Nov2020/LArPix/dataRuns/rootTrees/combined_with_light
 outputpath:  /home/lhep/PACMAN/larpix-analysis/lightCharge_anticorrelation
 datafiles ( 22 ): 
               sync_wLightReco_20201129_122204.root
               sync_wLightReco_20201129_122655.root
               sync_wLightReco_20201129_130449.root
               sync_wLightReco_20201129_134857.root
               sync_wLightReco_20201129_143427.root
               sync_wLightReco_20201129_150749.root
               sync_wLightReco_20201129_155227.root
               sync_wLightReco_20201129_162434.root
               sync_wLightReco_20201129_170918.root
               sync_wLightReco_20201129_174140.root
               sync_wLightReco_20201129_182214.root
               sync_wLightReco_20201129_185437.root
               sync_wLightReco_20201129_193550.root
               sync_wLightReco_20201129_200721.root
               sync_wLightReco_20201129_204607.root
               sync_wLightReco_202011

### Define voxelisation

In [4]:
# ============================================================
# Define voxelisation
# ============================================================
n_pixels_x = 70
n_pixels_y = 70
n_pixels_z = 70
pitch_x = 4.434
pitch_y = 4.434
pitch_z = 4.434
pixels_x_min = - pitch_x * n_pixels_x/2. #-155.19
pixels_x_max =   pitch_x * n_pixels_x/2. # 155.19
pixels_y_min = - pitch_y * n_pixels_y/2. #-155.19
pixels_y_max =   pitch_y * n_pixels_y/2. # 155.19
pixels_z_min = 0.
pixels_z_max = n_pixels_z*pitch_z*10 # factor 10 for correction... TODO: CHANGE THIS FOR NEXT PRODUCTION!!

### Define which plots to make

In [5]:
plot_all                                      = True

plot_chargeYield_vs_hitPosX                   = True
plot_chargeYield_vs_eField                    = True
plot_lightYield                               = False
plot_trackLength                              = False
plot_theta_vs_spread_x                        = False
plot_phi_vs_spread_x                          = False
plot_nHits                                    = False
plot_q_raw_tot                                = False
plot_q_corr_tot                               = False
plot_dQdl                                     = False
plot_startPosY_vs_startPosX                   = False
plot_endPosY_vs_endPosX                       = False
plot_startPosZ_vs_endPosZ                     = False
plot_nHits_vs_trackLength                     = False
plot_3D_graviCentre                           = False
plot_graviCentre_projection_xy                = False
plot_graviCentre_projection_xz                = False
plot_graviCentre_projection_yz                = False
plot_lightYield_vs_graviCentre_x              = False
plot_lightYield_vs_graviCentre_x_Selection    = False
plot_lightYield_vs_graviCentre_x_ProfilePlot  = False
plot_chargeYield_vs_eField                    = False
plot_chargeYield_vs_eField_ProfilePlot        = False
plot_chargeYield_vs_graviCentre_x             = False
plot_chargeYield_vs_graviCentre_x_Selection   = False
plot_chargeYield_vs_graviCentre_x_ProfilePlot = False

### Make a single TChain with input files

In [6]:
'''
#inputFileName = files[0]
inputFileName = 'sync_wLightReco_20201129_122204.root'
print(' Processing file', inputFileName)

input_tree = ROOT.TChain("t_out", "t_out")
input_tree.Add(datapath + '/' + inputFileName)
#for root_file in config["data_files"]:
#    input_tree.Add(root_file)
#input_tree.Add( "/path/to.root" )

# Turn on all branches of input tree
input_tree.SetBranchStatus("*",           1)
'''
print('Done.')

Done.


### Make List of TChains
Note: Make for every file a separate TChain.
Could substitute >1 files to a single TChain, but then it is more complicated to apply the electron lifetime correction...

In [7]:
#inputFileName = files[20]
#inputFileName = 'sync_wLightReco_20201129_122204.root'
#print(' Processing file', inputFileName)

input_tree_list       = []
eField_intensity_list = [] # [kV/cm]

'''
for i in range(10):
    input_tree = ROOT.TChain("t_out", "t_out")
    
    if i == 0: # 0.1 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201130_002722.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201130_005847.root')
        eField_intensity_list.append(0.1)
    if i == 1: # 0.2 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_230700.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_234000.root')
        eField_intensity_list.append(0.2)
    if i == 2: # 0.3 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_215550.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_222751.root')
        eField_intensity_list.append(0.3)
    if i == 3: # 0.4 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_204607.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_211819.root')
        eField_intensity_list.append(0.4)
    if i == 4: # 0.5 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_193550.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_200721.root')
        eField_intensity_list.append(0.5)
    if i == 5: # 0.6 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_182214.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_185437.root')
        eField_intensity_list.append(0.6)
    if i == 6: # 0.7 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_170918.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_174140.root')
        eField_intensity_list.append(0.7)
    if i == 7: # 0.8 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_155227.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_162434.root')
        eField_intensity_list.append(0.8)
    if i == 8: # 0.9 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_143427.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_150749.root')
        eField_intensity_list.append(0.9)
    if i == 9: # 1.0 kV/cm
        input_tree.Add(datapath + '/sync_wLightReco_20201129_122204.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_122655.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_130449.root')
        input_tree.Add(datapath + '/sync_wLightReco_20201129_134857.root')
        eField_intensity_list.append(1.0)
    
    # Turn on all branches of input tree
    input_tree.SetBranchStatus("*",           1)
    
    # Append input_tree TChain to the input_tree_list
    input_tree_list.append(input_tree)
'''

for i in range(len(files)):
    input_tree = ROOT.TChain("t_out", "t_out")
    #print(' files[',i,']: ', files[i])
    
    if files[i] == 'sync_wLightReco_20201130_002722.root' or\
       files[i] == 'sync_wLightReco_20201130_005847.root':
        input_tree.Add(datapath + '/' + files[i])
        eField_intensity_list.append(0.1)
    if files[i] == 'sync_wLightReco_20201129_230700.root' or\
       files[i] == 'sync_wLightReco_20201129_234000.root':
        input_tree.Add(datapath + '/' + files[i])
        eField_intensity_list.append(0.2)
    if files[i] == 'sync_wLightReco_20201129_215550.root' or\
       files[i] == 'sync_wLightReco_20201129_222751.root':
        input_tree.Add(datapath + '/' + files[i])
        eField_intensity_list.append(0.3)
    if files[i] == 'sync_wLightReco_20201129_204607.root' or\
       files[i] == 'sync_wLightReco_20201129_211819.root':
        input_tree.Add(datapath + '/' + files[i])
        eField_intensity_list.append(0.4)
    if files[i] == 'sync_wLightReco_20201129_193550.root' or\
       files[i] == 'sync_wLightReco_20201129_200721.root':
        input_tree.Add(datapath + '/' + files[i])
        eField_intensity_list.append(0.5)
    if files[i] == 'sync_wLightReco_20201129_182214.root' or\
       files[i] == 'sync_wLightReco_20201129_185437.root':
        input_tree.Add(datapath + '/' + files[i])
        eField_intensity_list.append(0.6)
    if files[i] == 'sync_wLightReco_20201129_170918.root' or\
       files[i] == 'sync_wLightReco_20201129_174140.root':
        input_tree.Add(datapath + '/' + files[i])
        eField_intensity_list.append(0.7)
    if files[i] == 'sync_wLightReco_20201129_155227.root' or\
       files[i] == 'sync_wLightReco_20201129_162434.root':
        input_tree.Add(datapath + '/' + files[i])
        eField_intensity_list.append(0.8)
    if files[i] == 'sync_wLightReco_20201129_143427.root' or\
       files[i] == 'sync_wLightReco_20201129_150749.root':
        input_tree.Add(datapath + '/' + files[i])
        eField_intensity_list.append(0.9)
    if files[i] == 'sync_wLightReco_20201129_122204.root' or\
       files[i] == 'sync_wLightReco_20201129_122655.root' or\
       files[i] == 'sync_wLightReco_20201129_130449.root' or\
       files[i] == 'sync_wLightReco_20201129_134857.root':
        input_tree.Add(datapath + '/' + files[i])
        eField_intensity_list.append(1.0)
    
    # Turn on all branches of input tree
    input_tree.SetBranchStatus("*",           1)
    
    # Append input_tree TChain to the input_tree_list
    input_tree_list.append(input_tree)
    
n_field_intensities = len(np.unique(eField_intensity_list))
print('Number of different field intensities: ', n_field_intensities)
    
print('List with input trees: ')
for i in range(len(input_tree_list)):
    print('   ', eField_intensity_list[i], 'kV/cm: \t [', input_tree_list[i], '] \t (', input_tree_list[i].GetEntries(), 'entries )')
    
print('Done.')

Number of different field intensities:  10
List with input trees: 
    1.0 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 1807 entries )
    1.0 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 19770 entries )
    1.0 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 19234 entries )
    1.0 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 19520 entries )
    0.9 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 19657 entries )
    0.9 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 19369 entries )
    0.8 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 19261 entries )
    0.8 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 18800 entries )
    0.7 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 19185 entries )
    0.7 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 18178 entries )
    0.6 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 18720 entries )
    0.6 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 18202 entries )
    0.5 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 18086 entries )
    0.5 kV/cm: 	 [ Name: t_out Title: t_out ] 	 ( 18149 entries )
    0.4 kV

### Create directories for plots

In [8]:
'''
# If make_plots: Create corresponding folder
make_plots = True

if make_plots:
    plot_folder = inputFileName[16:-5]
    os.system('rm -rf plots/' + str(plot_folder))
    os.system('mkdir plots/' + str(plot_folder))
    os.system('mkdir plots/' + str(plot_folder) + '/eventDisplays')
'''
for i in range(len(eField_intensity_list)):
    plot_folder = str(eField_intensity_list[i]) + '_kV_per_cm'
    os.system('rm -rf plots/' + str(plot_folder))
    os.system('mkdir plots/' + str(plot_folder))
    os.system('mkdir plots/' + str(plot_folder) + '/eventDisplays')
print('Done.')

Done.


### Define track selection cuts

In [9]:
# Define Selection Cuts
cut_ntracks      = 1    # Only consider events with number of reconstructed tracks == cut_ntracks
cut_lenght       = 250. # Track minimum length [mm]
cut_throughgoing = 130. # |max(track_start_y,track_end_y)| and |min(track_start_y,track_end_y)| must be greater than cut_throughgoing [mm] (tracks must start/end ad absolute y position > 130 mm, closer than ~2.5 cm from edge)
cut_phi          = 20.  # degrees (theta is +/- 90 deg if track is parallel to y-plane / x=const.)
#cut_nhits        = 50.  # Min. number of hits in the track [-]
#cut_residual2    = 3.   # Maximum allowed sqrt(res_x**2+res_y**2+res_z**2), in order to only allow tracks which are 'track-like' with a small spread

#TODO:
# CUT ON ANGLE (THETA?) -> THETA? MUST BE CLOSE TO ZERO IN ORDER TO HAVE ALMOST VERTICAL TRACKS

### Define electron drift velocities and lifetimes for every TChain

In [10]:
# Make lists with electron drift velocities and lifetimes
eDrift_vel     = [] # drift velocities [mm/us]
eLifetimes     = [] # one float for every TChain. Lifetime in [us]
eLifetimes_err = [] # one float for every TChain. Lifetime uncertainty in [us]

eLifetime_file = '../electron_lifetime/eLifetimes.txt'
try:
    file1 = open(eLifetime_file, 'r')
    lines = file1.readlines()
    file1.close()
except:
    print(' File '+str(eLifetime_file)+' not found...')

for chain in range(len(input_tree_list)):
    if   eField_intensity_list[chain] == 0.1: eDrift_vel.append(0.535)
    elif eField_intensity_list[chain] == 0.2: eDrift_vel.append(0.933)
    elif eField_intensity_list[chain] == 0.3: eDrift_vel.append(1.230)
    elif eField_intensity_list[chain] == 0.4: eDrift_vel.append(1.461)
    elif eField_intensity_list[chain] == 0.5: eDrift_vel.append(1.648)
    elif eField_intensity_list[chain] == 0.6: eDrift_vel.append(1.806)
    elif eField_intensity_list[chain] == 0.7: eDrift_vel.append(1.941)
    elif eField_intensity_list[chain] == 0.8: eDrift_vel.append(2.060)
    elif eField_intensity_list[chain] == 0.9: eDrift_vel.append(2.166)
    elif eField_intensity_list[chain] == 1.0: eDrift_vel.append(2.261)
    else:
        print(' WARNING: No eDrift velocity defined for eField intensity =', eField_intensity_list[chain], 'kV/cm ...')
        eDrift_vel.append(9999.9)
    
    eLifetime_assigned = False
    for line in lines:
        if eLifetime_assigned:
            break
        L = line.split(',')
        if str(L[0]) == str(files[chain][16:-5]):
            eLifetimes.append(float(L[1]))
            eLifetimes_err.append(float(L[2]))
            eLifetime_assigned = True
        #else:
        #    print(' WARNING: eLifetime for ', files[chain], ' not found in file ', eLifetime_file)
    if not eLifetime_assigned:
        print(' WARNING: eLifetime for file', files[chain], 'not found in file ', eLifetime_file)
        print('          --> Setting it to 4.0 us +/- 4.0 us ...')
        eLifetimes.append(4000.)
        eLifetimes_err.append(4000.)
    
    #eLifetimes.append(2.0) # TODO: Put here the proper lifetimes!
    
print('\n File summary: ')
for i in range(len(input_tree_list)):
    print('    file', i, ':\t', files[i],\
          '\t E-field: ', eField_intensity_list[i],\
          'kV/cm \t eLifetime: ', eLifetimes[i],\
          'ms \t eDrift: ', eDrift_vel[i], 'mm/us')    
    
print('Done.')

          --> Setting it to 4.0 us +/- 4.0 us ...

 File summary: 
    file 0 :	 sync_wLightReco_20201129_122204.root 	 E-field:  1.0 kV/cm 	 eLifetime:  1783.182472273415 ms 	 eDrift:  2.261 mm/us
    file 1 :	 sync_wLightReco_20201129_122655.root 	 E-field:  1.0 kV/cm 	 eLifetime:  2569.0812476631095 ms 	 eDrift:  2.261 mm/us
    file 2 :	 sync_wLightReco_20201129_130449.root 	 E-field:  1.0 kV/cm 	 eLifetime:  3361.5834110334017 ms 	 eDrift:  2.261 mm/us
    file 3 :	 sync_wLightReco_20201129_134857.root 	 E-field:  1.0 kV/cm 	 eLifetime:  2809.8163978796074 ms 	 eDrift:  2.261 mm/us
    file 4 :	 sync_wLightReco_20201129_143427.root 	 E-field:  0.9 kV/cm 	 eLifetime:  3117.3401241172055 ms 	 eDrift:  2.166 mm/us
    file 5 :	 sync_wLightReco_20201129_150749.root 	 E-field:  0.9 kV/cm 	 eLifetime:  2553.872420974916 ms 	 eDrift:  2.166 mm/us
    file 6 :	 sync_wLightReco_20201129_155227.root 	 E-field:  0.8 kV/cm 	 eLifetime:  2899.1377148192323 ms 	 eDrift:  2.06 mm/us
    file 7 :

### Loop over all TChains in input_tree_list

In [11]:
# Progress bar
from IPython.display import HTML, display

def progress(count, total, unit, message=''):
    return HTML("""
        <progress 
            value='{count}'
            max='{total}',
            style='width: 30%'
        >
            {count}
        </progress> {count}/{total} {unit} ({frac}%) ... {message}
    """.format(count=count, total=total, unit=unit, frac=int(float(count)/float(total)*100.),message=message))

print('Done.')

Done.


In [12]:
e_field_list            = []
track_length_list       = []
track_n_hits_list       = []
track_start_pos_x_list  = []
track_start_pos_y_list  = []
track_start_pos_z_list  = []
track_end_pos_x_list    = []
track_end_pos_y_list    = []
track_end_pos_z_list    = []
track_spread_x_list     = []
track_theta_list        = []
track_phi_list          = []
LCM_lightYield_list     = []
track_q_raw_tot_list    = []
track_q_corr_tot_list   = []
track_q_corr_in_GC_bin  = [] # containing n (=number of E-field intensities) lists with m (=number of gravicentres) q_corr values
pixels_chargeYield_list = [] # containing n (=number of E-field intensities) lists with m (=number of gravicentres) lists
pixels_hits_x_list      = []
pixels_hits_q_corr_list = []
track_cgx_list          = []
track_cgy_list          = []
track_cgz_list          = []


out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
start = time.time()
for eField in range(n_field_intensities):
    field_intensity = (eField+1)/10.
    
    #if not (field_intensity == 0.5 or field_intensity == 0.6):
    #    continue
    
    print( ' ----------------------------------------------------------------------------------- ')
    print(' Processing chains with field_intensity: ', field_intensity, 'kV/cm')
    
    # Define lists with interesting quantities
    track_length         = [] # Reconstructed track's length [mm]
    track_n_hits         = [] # Reconstructed track's total number of hits [-]
    track_start_pos_x    = [] # Reconstructed track's start position coordinates x [mm]
    track_start_pos_y    = [] # Reconstructed track's start position coordinates y [mm]
    track_start_pos_z    = [] # Reconstructed track's start position coordinates z [mm]
    track_end_pos_x      = [] # Reconstructed track's end position coordinates x [mm]
    track_end_pos_y      = [] # Reconstructed track's end position coordinates y [mm]
    track_end_pos_z      = [] # Reconstructed track's end position coordinates z [mm]
    track_spread_x       = [] # Reconstructed track's start_x-end_x [mm]
    track_theta          = [] # Reconstructed track's theta ('direction') [rad]
    track_phi            = [] # Reconstructed track's phi ('direction') [rad]
    LCM_lightYield       = [] # Total reconstructed light yield of LCM [p.e.]
    track_q_raw_tot      = [] # Reconstructed track's total charge deposit [ke] (no eLifetime correction)
    track_q_corr_tot     = [] # Reconstructed track's total charge deposit [ke] (eLifetime corrected)
    pixels_chargeYield   = [] # Reconstructed charge on pixels for every GC bin [ke] (eLifetime corrected)
    pixels_hits_x        = []
    pixels_hits_q_corr   = []
    track_cgx            = [] # gravicentre of the track (weighted by hits_q, but NOT YET ELECTRON LIFETIME CORRECTED -> TODO)
    track_cgy            = [] # gravicentre of the track (weighted by hits_q, but NOT YET ELECTRON LIFETIME CORRECTED -> TODO)
    track_cgz            = [] # gravicentre of the track (weighted by hits_q, but NOT YET ELECTRON LIFETIME CORRECTED -> TODO)
    
    for chain in range(len(input_tree_list)):
        
        # Skip the first two files (in order to have ~ the same statistics for all eField intensities):
        if chain == 0 or chain == 1:
            continue
            
        #if chain not in [17,21]:
        #    continue
        
        if eField_intensity_list[chain] != field_intensity:
            continue
        print('     Chain Nr.', chain)
        print('         eField [kV/cm]:     ', eField_intensity_list[chain])
        print('         eLifetime [ms]:     ', eLifetimes[chain])
        print('         eDrift_vel [mm/us]: ', eDrift_vel[chain])

        # Number of tracks in the TChain
        n_tracks = input_tree_list[chain].GetEntries()
        print('         Number of tracks : ', n_tracks)

        #print(input_tree_list[chain].Print())

        # List for events with a low light yield
        #low_light_events = []

        # List for track selection
        #selected_trackIDs = []

        accepted_tracks_counter = 0
        
        # Loop over all tracks in input_tree
        for track_id in range(n_tracks):
            if(track_id%5000==0):
                now = time.time()
                print('             Processed', track_id, 'of', n_tracks, 'tracks. \t Elapsed time:', math.ceil(now-start), ' seconds ... \r')
                
            input_tree_list[chain].GetEntry(track_id)

            # Apply Selection Cuts
            if input_tree_list[chain].t_track_length < cut_lenght:
                continue
            if input_tree_list[chain].t_event_ntracks > cut_ntracks:
                continue
            if abs(max(input_tree_list[chain].t_track_start_pos_y,input_tree_list[chain].t_track_end_pos_y)) < cut_throughgoing or\
               abs(min(input_tree_list[chain].t_track_start_pos_y,input_tree_list[chain].t_track_end_pos_y)) < cut_throughgoing:
                #print(input_tree_list[chain].t_track_start_pos_y)
                #print(input_tree_list[chain].t_track_end_pos_y)
                #print(' ---------------- ')
                continue
            
            if abs(input_tree_list[chain].t_track_phi*180./np.pi) < (90.-cut_phi) or abs(input_tree_list[chain].t_track_phi*180./np.pi) > (90.+cut_phi):
                continue

            #if track_id > 300:
            #    break

            #selected_trackIDs.append(input_tree_list[chain].t_trackID)

            #if track_id > 15:
            #    break

            '''        
            print(' ----------------------------------------- ')
            #print(' t_eventID:           ', input_tree_list[chain].t_eventID)
            print(' t_trackID:           ', input_tree_list[chain].t_trackID)
            #print(' t_event_q:           ', input_tree_list[chain].t_event_q)
            print(' t_track_length:      ', input_tree_list[chain].t_track_length)
            print(' t_event_ntracks:     ', input_tree_list[chain].t_event_ntracks)
            print(' t_track_start_pos_y: ', input_tree_list[chain].t_track_start_pos_y)
            print(' t_track_end_pos_y:   ', input_tree_list[chain].t_track_end_pos_y)
            #print(' t_track_q:           ', input_tree_list[chain].t_track_q)
            #print(' t_event_nhits:       ', input_tree_list[chain].t_event_nhits)
            #print(' t_track_nhits:       ', input_tree_list[chain].t_track_nhits)
            #print(' t_tlcm1_tot:         ', input_tree_list[chain].l_tlcm1_tot)
            '''

            track_length.append(input_tree_list[chain].t_track_length)
            track_n_hits.append(input_tree_list[chain].t_track_nhits)
            #track_q_tot.append(input_tree_list[chain].t_track_q) # this needs to be corrected by the electron lifetime (see below)

            track_start_pos_x.append(input_tree_list[chain].t_track_start_pos_x)
            track_start_pos_y.append(input_tree_list[chain].t_track_start_pos_y)
            track_start_pos_z.append(input_tree_list[chain].t_track_start_pos_z)
            track_end_pos_x.append(input_tree_list[chain].t_track_end_pos_x)
            track_end_pos_y.append(input_tree_list[chain].t_track_end_pos_y)
            track_end_pos_z.append(input_tree_list[chain].t_track_end_pos_z)
            
            track_spread_x.append(input_tree_list[chain].t_track_start_pos_x-input_tree_list[chain].t_track_end_pos_x)
            track_theta.append(input_tree_list[chain].t_track_theta)
            track_phi.append(input_tree_list[chain].t_track_phi)

            LCM_lightYield.append(input_tree_list[chain].l_tlcm1_tot)
            
            # For low light events
            #if input_tree_list[chain].l_tlcm1_tot<150:
            #    low_light_events.append(input_tree_list[chain].t_trackID)

            # Obtain the track's total charge and its center of gravity (weighted by deposited charge)
            # Loop over all hits
            q_raw_tot  = 0.
            q_corr_tot = 0.
            cgx        = 0.
            cgy        = 0.
            cgz        = 0.
            for hit in range(input_tree_list[chain].t_track_nhits):
                ### if hit>2:
                ###     break
                
                q_corr_factor = np.exp(0.1*input_tree_list[chain].t_track_hits_z[hit]/(eDrift_vel[chain]*eLifetimes[chain])) # TODO: factor 0.1 due to factor 10 problem -> fix it
                #    print(' hit_z [mm]:    ', 0.1 * input_tree_list[chain].t_track_hits_z[hit]) # TODO: factor 0.1 due to factor 10 problem -> fix it
                #    print(' q_corr_factor: ', q_corr_factor)
                q_raw_tot += input_tree_list[chain].t_track_hits_q[hit]
                hit_q_corr = input_tree_list[chain].t_track_hits_q[hit] * q_corr_factor # q_corr_factor corrects for the electron lifetime
                q_corr_tot += hit_q_corr
                
                ### print(' hit_x:  ', input_tree_list[chain].t_track_hits_x[hit], ' \t hit_q_corr: ', hit_q_corr)
                
                cgx += input_tree_list[chain].t_track_hits_x[hit]*hit_q_corr
                cgy += input_tree_list[chain].t_track_hits_y[hit]*hit_q_corr
                cgz += 0.1 * input_tree_list[chain].t_track_hits_z[hit]*hit_q_corr # TODO: factor 0.1 due to factor 10 problem -> fix it
                
                pixels_hits_x.append(input_tree_list[chain].t_track_hits_x[hit])
                pixels_hits_q_corr.append(hit_q_corr)
            
            track_q_raw_tot.append(q_raw_tot)
            track_q_corr_tot.append(q_corr_tot)
            pixels_chargeYield.append(q_corr_tot)
            # normalize gravicentre
            gravicentre_x = cgx/(q_corr_tot)
            gravicentre_y = cgy/(q_corr_tot)
            gravicentre_z = cgz/(q_corr_tot)
            track_cgx.append(gravicentre_x)
            track_cgy.append(gravicentre_y)
            track_cgz.append(gravicentre_z)
            #print(' ------------------------------ ')
            #print(' t_trackID:     ', input_tree_list[chain].t_trackID)
            #print(' cgx:           ', gravicentre_x)
            #print(' cgy:           ', gravicentre_y)
            #print(' cgz:           ', gravicentre_z)
            #print(' pixels_chargeYield:   ', pixels_chargeYield[-1])

    e_field_list.append(field_intensity)
    track_length_list.append(track_length)
    track_n_hits_list.append(track_n_hits)
    track_q_raw_tot_list.append(track_q_raw_tot)
    track_q_corr_tot_list.append(track_q_corr_tot)
    track_start_pos_x_list.append(track_start_pos_x)
    track_start_pos_y_list.append(track_start_pos_y)
    track_start_pos_z_list.append(track_start_pos_z)
    track_end_pos_x_list.append(track_end_pos_x)
    track_end_pos_y_list.append(track_end_pos_y)
    track_end_pos_z_list.append(track_end_pos_z)
    track_spread_x_list.append(track_spread_x)
    track_theta_list.append(track_theta)
    track_phi_list.append(track_phi)
    LCM_lightYield_list.append(LCM_lightYield)
    pixels_chargeYield_list.append(pixels_chargeYield)
    track_cgx_list.append(track_cgx)
    track_cgy_list.append(track_cgy)
    track_cgz_list.append(track_cgz)
    pixels_hits_x_list.append(pixels_hits_x)
    pixels_hits_q_corr_list.append(pixels_hits_q_corr)
    
    #print(' Number of selected tracks: ', len(selected_trackIDs))

    out.update(progress(eField+1,n_field_intensities,'E-field processed'))
out.update(progress(n_field_intensities,n_field_intensities,'E-field processed'))
print(' Done.')

### print(' pixels_hits_x_list:      ', pixels_hits_x_list)
### print(' pixels_hits_q_corr_list: ', pixels_hits_q_corr_list)


 ----------------------------------------------------------------------------------- 
 Processing chains with field_intensity:  0.1 kV/cm
     Chain Nr. 20
         eField [kV/cm]:      0.1
         eLifetime [ms]:      4000.0
         eDrift_vel [mm/us]:  0.535
         Number of tracks :  2068
             Processed 0 of 2068 tracks. 	 Elapsed time: 1  seconds ... 
     Chain Nr. 21
         eField [kV/cm]:      0.1
         eLifetime [ms]:      7638.208207029441
         eDrift_vel [mm/us]:  0.535
         Number of tracks :  1964
             Processed 0 of 1964 tracks. 	 Elapsed time: 1  seconds ... 
 ----------------------------------------------------------------------------------- 
 Processing chains with field_intensity:  0.2 kV/cm
     Chain Nr. 18
         eField [kV/cm]:      0.2
         eLifetime [ms]:      6553.752752610093
         eDrift_vel [mm/us]:  0.933
         Number of tracks :  7770
             Processed 0 of 7770 tracks. 	 Elapsed time: 1  seconds ... 
      

### Make Plots for Selected (!) Events

In [13]:
from plot_functions import *

In [14]:
# Plot charge yield per hit as a function of the hit position x

# Define how many graviCentre bins to have:
n_graviCentre_bins = 10
GC_binning = np.linspace(pixels_x_min,pixels_x_max,n_graviCentre_bins+1)

# Define binning
x_err  = []
x_vals = []
bin_width = GC_binning[1]-GC_binning[0]
for ibin in range(n_graviCentre_bins):
    x_err.append(0.5*bin_width)
    x_vals.append(GC_binning[ibin]+0.5*bin_width)

# Obtain chargeYield_perHit data
pixels_q_corr_tot_list        = []
pixels_q_corr_meanPerHit_list = []
pixels_q_corr_sem_list        = []
print(' Obtaining data...')
out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
for eField_index in range(n_field_intensities):    
    _q_corr_tot_list    = []
    _q_corr_summed     = []
    _q_corr_meanPerHit = []
    _q_corr_sem        = []
    for GC in range(n_graviCentre_bins):
        _q_corr_tot = []
        for hit in range(len(pixels_hits_x_list[eField_index])):
            if pixels_hits_x_list[eField_index][hit]>GC_binning[GC] and pixels_hits_x_list[eField_index][hit]<GC_binning[GC+1]:
                _q_corr_tot.append(pixels_hits_q_corr_list[eField_index][hit])
        _q_corr_summed.append(np.sum(_q_corr_tot))
        _q_corr_meanPerHit.append(np.mean(_q_corr_tot))
        _q_corr_sem.append(scipy.stats.sem(_q_corr_tot))
    pixels_q_corr_tot_list.append(_q_corr_summed)
    pixels_q_corr_meanPerHit_list.append(_q_corr_meanPerHit)
    pixels_q_corr_sem_list.append(_q_corr_sem)
    out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))

# Make plot
print(' Make plots...')
if plot_all or plot_chargeYield_vs_hitPosX:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(pixels_q_corr_meanPerHit_list[eField_index])
        if len(pixels_q_corr_meanPerHit_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        series_label_list = ['CY [ke]','GC x [mm] (scale factor: 1/10)']
        axis_labels = ['Hit Position x [mm]','Pixel Charge Yield per Hit [ke]']
        x_min,x_max = pixels_x_min, pixels_x_max
        y_min,y_max = 0,200
        save_name = 'plots/'+str(plot_folder)+'/'+'TGraphError_ChargeYield_vs_CoordX.png'
        plot_errorbars(x_vals,pixels_q_corr_meanPerHit_list[eField_index],x_err,pixels_q_corr_sem_list[eField_index],\
                       x_min,x_max,y_min,y_max,\
                       axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

 Obtaining data...


/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/numpy/core/_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
 

 Make plots...


/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:238: RuntimeWarning: Glyph 8722 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:201: RuntimeWarning: Glyph 8722 missing from current font.
  font.set_text(s, 0, flags=flags)


Done.


In [15]:
# Plot pixel charge yield per hit as a function of the E-field intensity

# Obtain chargeYield_perHit data
q_corr_meanPerHit = []
q_corr_sem        = []
for eField_index in range(n_field_intensities):    
    q_corr_meanPerHit.append(np.nanmean(pixels_hits_q_corr_list[eField_index]))
    q_corr_sem.append(scipy.stats.sem(pixels_hits_q_corr_list[eField_index]))

x_vals = np.linspace(0.1,1.0,10)
x_err = [0.05 for i in range(10)]

# Make plot
if plot_all or plot_chargeYield_vs_eField:
    x_min = 0
    x_max = 1.1
    y_min = 80
    y_max = 160
    axis_labels = ['E Field Intensity [kV/cm]','Pixel Charge Yield per Hit [ke]']
    save_name = 'plots/TGraphError_ChargeYield_vs_eField.png'
    plot_boxModel(x_vals,q_corr_meanPerHit,x_err,q_corr_sem,\
                   x_min,x_max,y_min,y_max,\
                   axis_labels,save_name)
print('Done.')

/home/roman/PACMAN/larpix-analysis/lightCharge_anticorrelation/plot_functions.py:652: RuntimeWarning: invalid value encountered in log
  return Q0 * E/beta * np.log(1.+beta/E)


 Q0:    155.40048759986396  +/-  1.7556709855115205
 beta:  0.10908766101861914  +/-  0.01034885228138031
Done.


In [16]:
# LCM_lightYield
if plot_all or plot_lightYield:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(LCM_lightYield_list[eField_index])
        if len(LCM_lightYield_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min  = np.min(LCM_lightYield_list[eField_index])
        x_max  = np.max(LCM_lightYield_list[eField_index])
        n_bins = 50
        axis_labels = ['Reconstructed LCM Light Yield [-]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h1_LCM_lightYield.png'
        plot_h1(LCM_lightYield_list[eField],x_min,x_max,n_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [17]:
# Track Length
if plot_all or plot_trackLength:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_length_list[eField_index])
        if len(track_length_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min  = 0
        x_max  = np.max(track_length_list[eField_index])
        n_bins = 50
        axis_labels = ['Track Length [mm]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h1_track_lengths.png'
        plot_h1(track_length_list[eField_index],x_min,x_max,n_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [18]:
# Track n_hits
if plot_all or plot_nHits:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_n_hits_list[eField_index])
        if len(track_n_hits_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min  = 0
        x_max  = np.max(track_n_hits_list[eField_index])
        n_bins = 50
        axis_labels = ['Number of Hits per Track [-]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h1_track_nhits.png'
        plot_h1(track_n_hits_list[eField_index],x_min,x_max,n_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [19]:
# Track q_raw_tot
if plot_all or plot_q_raw_tot:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_q_raw_tot_list[eField_index])
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min  = 0
        x_max  = 20000 #np.max(track_q_raw_tot_list[eField_index])
        n_bins = 50
        axis_labels = ['Total Deposited Charge (raw) [ke]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h1_track_q_raw_tot.png'
        plot_h1(track_q_raw_tot_list[eField_index],x_min,x_max,n_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [20]:
# Track q_corr_tot
if plot_all or plot_q_corr_tot:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_q_raw_tot_list[eField_index])
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min  = 0
        x_max  = 20000 #np.max(track_q_corr_tot_list[eField_index])
        n_bins = 50
        axis_labels = ['Total Deposited Charge (e-lifetime corrected) [ke]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h1_track_q_corr_tot.png'
        plot_h1(track_q_corr_tot_list[eField_index],x_min,x_max,n_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [21]:
# Track dQ/dl (eLifetime corrected)
if plot_all or plot_dQdl:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(pixels_chargeYield_list[eField_index])
        if len(pixels_chargeYield_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min  = 0
        #x_max  = np.max(np.array(track_q_raw_tot_list[eField_index])/np.array(track_length_list[eField_index]))
        x_max  = np.max(np.array(pixels_chargeYield_list[eField_index])/np.array(track_length_list[eField_index]))
        n_bins = 50
        axis_labels = ['<dQ/dl> [ke/mm]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h1_track_dQ_dl_eLifetimeCorrected.png'
        #plot_h1(np.array(track_q_raw_tot_list[eField_index])/np.array(track_length_list[eField_index]),x_min,x_max,n_bins,axis_labels,save_name)
        #plot_h1(np.array(pixels_chargeYield_list[eField_index])/np.array(track_length_list[eField_index]),x_min,x_max,n_bins,axis_labels,save_name)
        plot_h1(np.array(track_q_corr_tot_list[eField_index])/np.array(track_length_list[eField_index]),x_min,x_max,n_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [22]:
# start_pos_y vs. start_pos_x
if plot_all or plot_startPosY_vs_startPosX:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_start_pos_x_list[eField_index])
        if len(track_start_pos_x_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min    = pixels_x_min # np.min(track_start_pos_x_list[eField_index])
        x_max    = pixels_x_max # np.max(track_start_pos_x_list[eField_index])
        y_min    = pixels_y_min # np.min(track_start_pos_y_list[eField_index])
        y_max    = pixels_y_max # np.max(track_start_pos_y_list[eField_index])
        n_bins_x = n_pixels_x
        n_bins_y = n_pixels_y
        x_bins = np.linspace(x_min,x_max,n_bins_x)
        y_bins = np.linspace(y_min,y_max,n_bins_y)
        #x_bins = np.arange(x_min,x_max+1,step=1.)
        #y_bins = np.arange(y_min,y_max+1,step=1.)
        axis_labels = ['Track Start Position x [mm]','Track Start Position y [mm]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h2_track_startPos.png'
        plot_h2([track_start_pos_x_list[eField_index],track_start_pos_y_list[eField_index]],x_bins,y_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:238: RuntimeWarning: Glyph 8722 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/matplotlib/backends/backend_agg.py:201: RuntimeWarning: Glyph 8722 missing from current font.
  font.set_text(s, 0, flags=flags)


Done.


In [23]:
# end_pos_y vs. end_pos_x
if plot_all or plot_endPosY_vs_endPosX:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_end_pos_x_list[eField_index])
        if len(track_end_pos_x_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min    = pixels_x_min # np.min(track_start_pos_x_list[eField_index])
        x_max    = pixels_x_max # np.max(track_start_pos_x_list[eField_index])
        y_min    = pixels_y_min # np.min(track_start_pos_y_list[eField_index])
        y_max    = pixels_y_max # np.max(track_start_pos_y_list[eField_index])
        n_bins_x = n_pixels_x
        n_bins_y = n_pixels_y
        x_bins = np.linspace(x_min,x_max,n_bins_x)
        y_bins = np.linspace(y_min,y_max,n_bins_y)
        #x_bins = np.arange(x_min,x_max+1,step=1.)
        #y_bins = np.arange(y_min,y_max+1,step=1.)
        axis_labels = ['Track End Position x [mm]','Track End Position y [mm]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h2_track_endPos.png'
        plot_h2([track_end_pos_x_list[eField_index],track_end_pos_y_list[eField_index]],x_bins,y_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [24]:
# start_pos_z vs. end_pos_z
if plot_all or plot_startPosZ_vs_endPosZ:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_start_pos_x_list[eField_index])
        if len(track_start_pos_z_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        z_start_min = np.min(track_start_pos_z_list[eField_index])
        z_start_max = np.max(track_start_pos_z_list[eField_index])
        z_end_min   = np.min(track_end_pos_z_list[eField_index])
        z_end_max   = np.max(track_end_pos_z_list[eField_index])
        n_bins = n_pixels_z
        x_bins = np.linspace(z_start_min,z_start_max,n_bins)
        y_bins = np.linspace(z_end_min,z_end_max,n_bins)
        #x_bins = np.arange(x_min,x_max+1,step=1.)
        #y_bins = np.arange(y_min,y_max+1,step=1.)
        axis_labels = ['Track Start Position z [mm]','Track End Position z [mm]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h2_trEndz_vs_trStartz.png'
        plot_h2([track_start_pos_z_list[eField_index],track_end_pos_z_list[eField_index]],x_bins,y_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [25]:
# NHits vs. TrackLength
if plot_all or plot_nHits_vs_trackLength:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_length_list[eField_index])
        if len(track_length_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min    = 200
        x_max    = 400
        y_min    = 0
        y_max    = 150
        n_bins_x = 50
        n_bins_y = 50
        x_bins = np.linspace(x_min,x_max,n_bins_x)
        y_bins = np.linspace(y_min,y_max,n_bins_y)
        #x_bins = np.arange(x_min,x_max+1,step=1.)
        #y_bins = np.arange(y_min,y_max+1,step=1.)
        axis_labels = ['Track Length [mm]','Number of Hits per Track [-]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h2_trackLength_vs_trackNHits.png'
        plot_h2([track_length_list[eField_index],track_n_hits_list[eField_index]],x_bins,y_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [26]:
# Theta [rad] vs. Spread_x (spread_x = track_start_pos_x - track_end_pos_x)
if plot_all or plot_theta_vs_spread_x:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_theta_list[eField_index])
        if len(track_theta_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min    = pixels_x_min
        x_max    = pixels_x_max
        y_min    = -np.pi
        y_max    = np.pi
        n_bins_x = n_pixels_x
        n_bins_y = 90
        x_bins = np.linspace(x_min,x_max,n_bins_x)
        y_bins = np.linspace(y_min,y_max,n_bins_y)
        #x_bins = np.arange(x_min,x_max+1,step=1.)
        #y_bins = np.arange(y_min,y_max+1,step=1.)
        axis_labels = ['Track start_x - end_x [mm]','Track Theta [rad]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h2_theta_rad_vs_spread_x.png'
        plot_h2([track_spread_x_list[eField_index],track_theta_list[eField_index]],x_bins,y_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [27]:
# Theta [deg] vs. Spread_x (spread_x = track_start_pos_x - track_end_pos_x)
if plot_all or plot_theta_vs_spread_x:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_theta_list[eField_index])
        if len(track_theta_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min    = pixels_x_min
        x_max    = pixels_x_max
        y_min    = 70
        y_max    = 110
        n_bins_x = n_pixels_x
        n_bins_y = 40
        x_bins = np.linspace(x_min,x_max,n_bins_x)
        y_bins = np.linspace(y_min,y_max,n_bins_y)
        #x_bins = np.arange(x_min,x_max+1,step=1.)
        #y_bins = np.arange(y_min,y_max+1,step=1.)
        axis_labels = ['Track start_x - end_x [mm]','Track Theta [deg]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h2_theta_deg_vs_spread_x.png'
        plot_h2([track_spread_x_list[eField_index],np.array(track_theta_list[eField_index])*180./np.pi],x_bins,y_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [28]:
# Phi [rad] vs. Spread_x (spread_x = track_start_pos_x - track_end_pos_x)
if plot_all or plot_phi_vs_spread_x:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_phi_list[eField_index])
        if len(track_phi_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min    = pixels_x_min
        x_max    = pixels_x_max
        y_min    = -np.pi
        y_max    = np.pi
        n_bins_x = n_pixels_x
        n_bins_y = 90
        x_bins = np.linspace(x_min,x_max,n_bins_x)
        y_bins = np.linspace(y_min,y_max,n_bins_y)
        #x_bins = np.arange(x_min,x_max+1,step=1.)
        #y_bins = np.arange(y_min,y_max+1,step=1.)
        axis_labels = ['Track start_x - end_x [mm]','Track Phi [rad]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h2_phi_rad_vs_spread_x.png'
        plot_h2([track_spread_x_list[eField_index],track_phi_list[eField_index]],x_bins,y_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [29]:
# abs(Phi) [deg] vs. Spread_x (spread_x = track_start_pos_x - track_end_pos_x)
if plot_all or plot_phi_vs_spread_x:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_phi_list[eField_index])
        if len(track_phi_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min    = pixels_x_min
        x_max    = pixels_x_max
        y_min    = 0.
        y_max    = 180.
        n_bins_x = n_pixels_x
        n_bins_y = 90
        x_bins = np.linspace(x_min,x_max,n_bins_x)
        y_bins = np.linspace(y_min,y_max,n_bins_y)
        #x_bins = np.arange(x_min,x_max+1,step=1.)
        #y_bins = np.arange(y_min,y_max+1,step=1.)
        axis_labels = ['Track start_x - end_x [mm]','Track |Phi| [deg]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h2_phi_deg_vs_spread_x.png'
        plot_h2([track_spread_x_list[eField_index],abs(np.array(track_phi_list[eField_index])*180./np.pi)],x_bins,y_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [30]:
# 3D plot of gravicentres
if plot_all or plot_3D_graviCentre:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_cgx_list[eField_index])
        if len(track_cgx_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min    = pixels_x_min
        x_max    = pixels_x_max
        y_min    = pixels_y_min
        y_max    = pixels_y_max
        z_min    = 0
        z_max    = 600 #max(pixels_z_max,np.max(track_cgz_list[eField_index]))
        n_bins_x = n_pixels_x
        n_bins_y = n_pixels_y
        n_bins_z = n_pixels_z
        x_bins = np.linspace(x_min,x_max,n_bins_x)
        y_bins = np.linspace(y_min,y_max,n_bins_y)
        z_bins = np.linspace(z_min,z_max,n_bins_z)
        save_name = 'plots/'+str(plot_folder)+'/'+'h3_graviCentre.png'
        plot_h3_unweighted([track_cgx_list[eField_index],track_cgy_list[eField_index],track_cgz_list[eField_index]],\
                           x_bins,y_bins,z_bins,['Coordinate x [mm]','Coordinate y [mm]','Coordinate z [mm]'],save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [31]:
# Gravycentre projection onto xy plane
if plot_all or plot_graviCentre_projection_xy:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_cgx_list[eField_index])
        if len(track_cgx_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min    = pixels_x_min
        x_max    = pixels_x_max
        y_min    = pixels_y_min
        y_max    = pixels_y_max
        n_bins_x = n_pixels_x
        n_bins_y = n_pixels_y
        x_bins = np.linspace(x_min,x_max,n_bins_x)
        y_bins = np.linspace(y_min,y_max,n_bins_y)
        #x_bins = np.arange(x_min,x_max+1,step=1.)
        #y_bins = np.arange(y_min,y_max+1,step=1.)
        axis_labels = ['Coordinate x [mm]','Coordinate y [mm]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h2_graviCentre_xy.png'
        plot_h2([track_cgx_list[eField_index],track_cgy_list[eField_index]],x_bins,y_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [32]:
# Gravycentre projection onto xz plane
if plot_all or plot_graviCentre_projection_xz:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_cgx_list[eField_index])
        if len(track_cgx_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min    = pixels_x_min
        x_max    = pixels_x_max
        z_min    = 0
        z_max    = max(pixels_z_max,np.max(track_cgz_list[eField_index]))
        n_bins_x = n_pixels_x
        n_bins_z = n_pixels_z
        x_bins = np.linspace(x_min,x_max,n_bins_x)
        z_bins = np.linspace(z_min,z_max,n_bins_z)
        #x_bins = np.arange(x_min,x_max+1,step=1.)
        #y_bins = np.arange(y_min,y_max+1,step=1.)
        axis_labels = ['Coordinate x [mm]','Coordinate z [mm]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h2_graviCentre_xz.png'
        plot_h2([track_cgx_list[eField_index],track_cgz_list[eField_index]],x_bins,z_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [33]:
# Gravycentre projection onto yz plane
if plot_all or plot_graviCentre_projection_yz:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_cgy_list[eField_index])
        if len(track_cgy_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        y_min    = pixels_y_min
        y_max    = pixels_y_max
        z_min    = 0
        z_max    = max(pixels_z_max,np.max(track_cgz_list[eField_index]))
        n_bins_y = n_pixels_y
        n_bins_z = n_pixels_z
        y_bins = np.linspace(y_min,y_max,n_bins_y)
        z_bins = np.linspace(z_min,z_max,n_bins_z)
        #y_bins = np.arange(y_min,y_max+1,step=1.)
        #z_bins = np.arange(z_min,z_max+1,step=1.)
        axis_labels = ['Coordinate y [mm]','Coordinate z [mm]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h2_graviCentre_yz.png'
        plot_h2([track_cgy_list[eField_index],track_cgz_list[eField_index]],y_bins,z_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [34]:
# LCM light-yield vs. track gravicentre_x
if plot_all or plot_lightYield_vs_graviCentre_x:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(track_cgx_list[eField_index])
        if len(track_cgx_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min    = pixels_x_min
        x_max    = pixels_x_max
        y_min    = 0 # np.min(LCM_lightYield_list[eField_index])
        y_max    = 1500 # np.max(LCM_lightYield_list[eField_index])
        n_bins_x = n_pixels_x
        n_bins_y = n_pixels_y
        x_bins = np.linspace(x_min,x_max,n_bins_x)
        y_bins = np.linspace(y_min,y_max,n_bins_y)
        axis_labels = ['Track Gravicentre x [mm]','LCM Light Yield [p.e.]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h2_LCM_lightYield_vs_trackGC.png'
        plot_h2([track_cgx_list[eField_index],LCM_lightYield_list[eField_index]],x_bins,y_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
print('Done.')

Done.


In [35]:
# LCM light-yield vs. track gravicentre_x, all with selection cut line, and only selected with selection cut line
if plot_all or plot_lightYield_vs_graviCentre_x_Selection:
    x_min = pixels_x_min
    x_max = pixels_x_max
    y_min = 0
    y_max = 1500
    #for i in range(len(LCM_lightYield_list)):
    #    if np.max(LCM_lightYield_list[i]) > y_max:
    #        y_max = np.max(LCM_lightYield_list[i])
    n_bins_x = n_pixels_x
    n_bins_y = n_pixels_y
    x_bins = np.linspace(x_min,x_max,n_bins_x)
    y_bins = np.linspace(y_min,y_max,n_bins_y)
    axis_labels = ['Track Gravicentre x [mm]','LCM Light Yield [p.e.]','Entries [-]']

    LY_cut_at_xmin = 400 # min. light yield at x_min (near end)
    LY_cut_at_xmax = 50  # min. light yield at x_max (far end)

    # Plot only selected
    print('plotting selection...')
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        save_name = 'plots/'+str(plot_folder)+'/h2_LCM_lightYield_vs_trackGC_selected.png'
        gc_list = []
        ly_list = []
        for entry in range(len(LCM_lightYield_list[eField_index])):
            if LCM_lightYield_list[eField_index][entry]>-(track_cgx_list[eField_index][entry]-x_min)*(LY_cut_at_xmin-LY_cut_at_xmax)/(x_max-x_min)+LY_cut_at_xmin:
                gc_list.append(track_cgx_list[eField_index][entry])
                ly_list.append(LCM_lightYield_list[eField_index][entry])
        plot_LY_vs_GCx_selection([gc_list,ly_list],x_bins,y_bins,axis_labels,save_name,LY_cut_at_xmin,LY_cut_at_xmax)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))

    # Plot all
    print('plotting all...')
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        save_name = 'plots/'+str(plot_folder)+'/h2_LCM_lightYield_vs_trackGC_all.png'
        gc_list = []
        ly_list = []
        for entry in range(len(LCM_lightYield_list[eField_index])):
            if True:
                gc_list.append(track_cgx_list[eField_index][entry])
                ly_list.append(LCM_lightYield_list[eField_index][entry])
        plot_LY_vs_GCx_selection([gc_list,ly_list],x_bins,y_bins,axis_labels,save_name,LY_cut_at_xmin,LY_cut_at_xmax)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))

print('Done.')

plotting selection...


plotting all...


Done.


In [36]:
# Profile plot of LCM light-yield vs. track gravicentre_x
if plot_all or plot_lightYield_vs_graviCentre_x_ProfilePlot:
    x_min    = pixels_x_min
    x_max    = pixels_x_max
    n_bins_x = n_graviCentre_bins
    x_width  = (x_max-x_min)/n_bins_x
    x_err    = [0.5*x_width for i in range(n_bins_x)]
    y_min    = 0
    y_max    = 1500
    #for i in range(len(LCM_lightYield_list)):
    #    if np.max(LCM_lightYield_list[i]) > y_max:
    #        y_max = np.max(LCM_lightYield_list[i])

    # Define lists which will be filled with n (=number of different eField-intensities) lists with info about
    # gravicentre, mean light yield and standard deviation of the mean light yield
    gc_bin_i_list      = []
    ly_mean_bin_i_list = []
    ly_std_bin_i_list  = []
    ly_sem_bin_i_list  = []
    plot_folder_list   = []

    # Loop over all TChains and obtain data for the profile plots
    for eField_index in range(n_field_intensities):
        #print(track_cgx_list[eField_index])
        gc_bin_i      = []
        ly_mean_bin_i = []
        ly_std_bin_i  = []
        ly_sem_bin_i  =[]
        # Loop over all bins
        for bin_i in range(n_bins_x):
            gc_bin_i.append((bin_i+0.5)*x_width+x_min)
            ly_in_bin_i = []
            # Loop over all TChains
            for entry in range(len(LCM_lightYield_list[eField_index])):
                # Check if GC is in current bin
                if track_cgx_list[eField_index][entry]>((bin_i)*x_width+x_min) and track_cgx_list[eField_index][entry]<((bin_i+1)*x_width+x_min):
                    # Check if LY is not below selection criterium
                    if LCM_lightYield_list[eField_index][entry]>-(track_cgx_list[eField_index][entry]-x_min)*(LY_cut_at_xmin-LY_cut_at_xmax)/(x_max-x_min)+LY_cut_at_xmin:
                        ly_in_bin_i.append(LCM_lightYield_list[eField_index][entry])
            ly_mean_bin_i.append(np.mean(ly_in_bin_i))
            ly_std_bin_i.append(np.std(ly_in_bin_i))
            ly_sem_bin_i.append(scipy.stats.sem(ly_in_bin_i))
        gc_bin_i_list.append(gc_bin_i)
        ly_mean_bin_i_list.append(ly_mean_bin_i)
        ly_std_bin_i_list.append(ly_std_bin_i)
        ly_sem_bin_i_list.append(ly_sem_bin_i)
        plot_folder_list.append(str(e_field_list[eField_index]) + '_kV_per_cm')

    # Don't plot points with sem = 0 (likely points with only 1 measurement):
    for entry1 in range(len(ly_sem_bin_i_list)):
        for entry2 in range(len(ly_sem_bin_i_list[entry1])):
            if ly_sem_bin_i_list[entry1][entry2]==0:
                ly_mean_bin_i_list[entry1][entry2]=float('nan')

    restrict_to_entries = [2,5,9]
    
    axis_labels = ['Track Gravicentre x [mm]','LCM Light Yield [p.e.]']
    save_name = 'plots/profile2D_LCM_lightYield_vs_trackGC.png'
    plot_profile_of_2D_hist_00(gc_bin_i_list,\
                            ly_mean_bin_i_list,\
                            x_err,\
                            ly_sem_bin_i_list,\
                            plot_folder_list,\
                            axis_labels,\
                            x_min,\
                            x_max,\
                            y_min,\
                            y_max,\
                            save_name,\
                            restrict_to_entries)
print('Done.')

/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/numpy/core/_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/home/roman/software/anaconda3/envs/larpix-analysis-38/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
 

Done.


In [37]:
# pixel chargeYield vs. E-field intensity
if plot_all or plot_chargeYield_vs_eField:
    chargeYield_values = []
    eField_values      = []
    for eField_index in range(n_field_intensities):
        #print(pixels_chargeYield_list[eField_index])
        if len(pixels_chargeYield_list[eField_index]) == 0: continue
        for entry in range(len(pixels_chargeYield_list[eField_index])):
            chargeYield_values.append(pixels_chargeYield_list[eField_index][entry])
            eField_values.append((eField_index+1)/10.)
        #print((eField_index+1)/10.)
    x_min    = -0.05
    x_max    = 1.05
    y_min    = 4000  # np.min(chargeYield_values)
    y_max    = 15000 # np.max(chargeYield_values)
    n_bins_x = 10+2
    n_bins_y = 50
    x_bins = np.linspace(x_min,x_max,n_bins_x)
    y_bins = np.linspace(y_min,y_max,n_bins_y)
    axis_labels = ['E Field Intensity [kV/cm]','Total Pixel Charge Yield [ke]','Entries [-]']
    save_name = 'plots/'+'h2_pixel_chargeYield_vs_eField.png'
    plot_h2([eField_values,chargeYield_values],x_bins,y_bins,axis_labels,save_name)
print('Done.')

Done.


In [38]:
# Profile plot of pixel charge-yield vs. eField
if plot_all or plot_chargeYield_vs_eField_ProfilePlot:
    x_min    = -0.05
    x_max    = 1.05
    y_min    = 6000  # np.min(chargeYield_values)
    y_max    = 10000 # np.max(chargeYield_values)
    
    chargeYield_mean = []
    chargeYield_std  = []
    chargeYield_sem  = []
    eField_vals      = []
    #eField_errors    = []
    x_err = 0.05
    plot_folder_list = []
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(pixels_chargeYield_list[eField_index])
        if len(pixels_chargeYield_list[eField_index]) == 0: continue
        
        CY_values = []
        for entry in range(len(pixels_chargeYield_list[eField_index])):
            CY_values.append(pixels_chargeYield_list[eField_index][entry])
        
        chargeYield_mean.append(np.mean(CY_values))
        chargeYield_std.append(np.std(CY_values))
        chargeYield_sem.append(scipy.stats.sem(CY_values))
        eField_vals.append((eField_index+1)/10.)
        #eField_errors.append(0.05)
        plot_folder_list.append(str(e_field_list[eField_index]) + '_kV_per_cm')
    
    axis_labels = ['E Field Intensity [kV/cm]','Total Pixel Charge Yield [ke]']
    save_name = 'plots/profile2D_pixel_chargeYield_vs_eField.png'
    plot_legend = False
    plot_profile_of_2D_hist_01(eField_vals,\
                            chargeYield_mean,\
                            x_err,\
                            chargeYield_sem,\
                            axis_labels,\
                            x_min,\
                            x_max,\
                            y_min,\
                            y_max,\
                            save_name)
    out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
#'''
print('Done.')

Done.


In [39]:
# This plot will be the same as the first one in the next cell
'''
# pixel chargeYield vs. track gravicentre_x
if plot_all or plot_chargeYield_vs_graviCentre_x or True:
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        #print(pixels_chargeYield_list[eField_index])
        if len(pixels_chargeYield_list[eField_index]) == 0: continue
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        x_min    = pixels_x_min
        x_max    = pixels_x_max
        y_min    = 0 # np.min(pixels_chargeYield_list[eField_index])
        y_max    = 20000 #np.max(pixels_chargeYield_list[eField_index])
        n_bins_x = n_pixels_x
        n_bins_y = n_pixels_y
        x_bins = np.linspace(x_min,x_max,n_bins_x)
        y_bins = np.linspace(y_min,y_max,n_bins_y)
        axis_labels = ['Track Gravicentre x [mm]','Total Pixel Charge Yield [ke]','Entries [-]']
        save_name = 'plots/'+str(plot_folder)+'/'+'h2_pixel_chargeYield_vs_trackGC.png'
        plot_h2([track_cgx_list[eField_index],pixels_chargeYield_list[eField_index]],x_bins,y_bins,axis_labels,save_name)
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))
'''
print('Done.')

Done.


In [40]:
# Pixel charge-yield vs. track gravicentre_x, all and in a second plot only those which passed the lightYield selection
if plot_all or plot_chargeYield_vs_graviCentre_x_Selection:
    x_min    = pixels_x_min
    x_max    = pixels_x_max
    y_min    = 0 # np.min(pixels_chargeYield_list[eField_index])
    y_max    = 20000
    #for i in range(len(pixels_chargeYield_list)):
    #    if np.max(pixels_chargeYield_list[i]) > y_max:
    #        y_max = np.max(pixels_chargeYield_list[i])
    n_bins_x = n_pixels_x
    n_bins_y = n_pixels_y
    x_bins = np.linspace(x_min,x_max,n_bins_x)
    y_bins = np.linspace(y_min,y_max,n_bins_y)
    axis_labels = ['Track Gravicentre x [mm]','Total Pixel Charge Yield [ke]','Entries [-]']

    LY_cut_at_xmin = 400 # min. light yield at x_min (near end)
    LY_cut_at_xmax = 50  # min. light yield at x_max (far end)

    # Plot only those passing the lightYield selection
    print('plotting selection...')
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        save_name = 'plots/'+str(plot_folder)+'/h2_pixel_chargeYield_vs_trackGC_selected.png'
        gc_list = []
        cy_list = []
        for entry in range(len(LCM_lightYield_list[eField_index])):
            if LCM_lightYield_list[eField_index][entry]>-(track_cgx_list[eField_index][entry]-x_min)*(LY_cut_at_xmin-LY_cut_at_xmax)/(x_max-x_min)+LY_cut_at_xmin:
                gc_list.append(track_cgx_list[eField_index][entry])
                cy_list.append(pixels_chargeYield_list[eField_index][entry])
        plot_LY_vs_GCx_selection([gc_list,cy_list],x_bins,y_bins,axis_labels,save_name,-1,-1) # Note: Use the same function as for the lightYield
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))

    # Plot all
    print('plotting all...')
    out = display(progress(0,n_field_intensities,'E-field intensities processed'),display_id=True)
    for eField_index in range(n_field_intensities):
        plot_folder = str(e_field_list[eField_index]) + '_kV_per_cm'
        #print(' plot_folder: ', plot_folder)
        save_name = 'plots/'+str(plot_folder)+'/h2_pixel_chargeYield_vs_trackGC_all.png'
        gc_list = []
        cy_list = []
        for entry in range(len(LCM_lightYield_list[eField_index])):
            if True:
                gc_list.append(track_cgx_list[eField_index][entry])
                cy_list.append(pixels_chargeYield_list[eField_index][entry])
        plot_LY_vs_GCx_selection([gc_list,cy_list],x_bins,y_bins,axis_labels,save_name,-1,-1) # Note: Use the same function as for the lightYield
        out.update(progress(eField_index+1,n_field_intensities,'E-field intensities processed'))
    out.update(progress(n_field_intensities,n_field_intensities,'E-field intensities processed'))

print('Done.')

plotting selection...


plotting all...


Done.


In [41]:
# Profile plot of pixel charge-yield vs. track gravicentre_x
if plot_all or plot_chargeYield_vs_graviCentre_x_ProfilePlot:
    x_min    = pixels_x_min
    x_max    = pixels_x_max
    n_bins_x = n_graviCentre_bins
    x_width  = (x_max-x_min)/n_bins_x
    x_err    = [0.5*x_width for i in range(n_bins_x)]
    y_min    = 6000
    y_max    = 10000
    #for i in range(len(pixels_chargeYield_list)):
    #    if np.max(pixels_chargeYield_list[i]) > y_max:
    #        y_max = np.max(pixels_chargeYield_list[i])

    # Define lists which will be filled with n (=number of different eField-intensities) lists with info about
    # gravicentre, mean charge yield and standard deviation of the mean charge yield
    gc_bin_i_list      = []
    cy_mean_bin_i_list = []
    cy_std_bin_i_list  = [] # standard deviation
    cy_sem_bin_i_list  = [] # standard error of the mean
    plot_folder_list   = []

    # Loop over all TChains and obtain data for the profile plots
    for eField_index in range(n_field_intensities):
        #print(track_cgx_list[eField_index])
        gc_bin_i      = []
        cy_mean_bin_i = []
        cy_std_bin_i  = []
        cy_sem_bin_i  = []
        # Loop over all bins
        for bin_i in range(n_bins_x):
            gc_bin_i.append((bin_i+0.5)*x_width+x_min)
            cy_in_bin_i = []
            # Loop over all TChains
            for entry in range(len(pixels_chargeYield_list[eField_index])):
                # Check if GC is in current bin
                if track_cgx_list[eField_index][entry]>((bin_i)*x_width+x_min) and track_cgx_list[eField_index][entry]<((bin_i+1)*x_width+x_min):
                    # Check if LY (not CY!) is not below selection criterium
                    if LCM_lightYield_list[eField_index][entry]>-(track_cgx_list[eField_index][entry]-x_min)*(LY_cut_at_xmin-LY_cut_at_xmax)/(x_max-x_min)+LY_cut_at_xmin:
                        cy_in_bin_i.append(pixels_chargeYield_list[eField_index][entry])
            cy_mean_bin_i.append(np.mean(cy_in_bin_i))
            cy_std_bin_i.append(np.std(cy_in_bin_i))
            cy_sem_bin_i.append(scipy.stats.sem(cy_in_bin_i))
        gc_bin_i_list.append(gc_bin_i)
        cy_mean_bin_i_list.append(cy_mean_bin_i)
        cy_std_bin_i_list.append(cy_std_bin_i)
        cy_sem_bin_i_list.append(cy_sem_bin_i)
        plot_folder_list.append(str(e_field_list[eField_index]) + '_kV_per_cm')
        
    restrict_to_entries = [2,5,9]

    axis_labels = ['Track Gravicentre x [mm]','Total Pixel Charge Yield [ke]']
    save_name = 'plots/profile2D_pixel_chargeYield_vs_trackGC.png'
    plot_profile_of_2D_hist_00(gc_bin_i_list,\
                            cy_mean_bin_i_list,\
                            x_err,\
                            cy_sem_bin_i_list,\
                            plot_folder_list,\
                            axis_labels,\
                            x_min,\
                            x_max,\
                            y_min,\
                            y_max,\
                            save_name,\
                            restrict_to_entries)
print('Done.')

Done.


In [42]:
# TGraphError plot of light-yield and charge-yield vs. eField intensity
'''
np.set_printoptions(precision=2)
for eField in range(len(cy_mean_bin_i_list)): #(len(cy_mean_bin_i_list)):
    #if eField not in [1,5,9]:
    #    continue
    print(' ------------------------------------------------------------------------------- ')
    print(' eField [kV/cm]: ', (eField+1)/10.)
    print(' At this field, have the following bins: ')
    #print('     x-bins:    ', np.array(gc_bin_i_list[eField]))
    print('     ly_mean:   ', np.array(ly_mean_bin_i_list[eField]))
    print('     ly_std:    ', np.array(ly_std_bin_i_list[eField]))
    print('     ly_sem:    ', np.array(ly_sem_bin_i_list[eField]))
    #print('     cy_mean:   ', np.array(cy_mean_bin_i_list[eField]))
    #print('     cy_std:    ', np.array(cy_std_bin_i_list[eField]))
    #print('     cy_sem:    ', np.array(cy_sem_bin_i_list[eField]))
    #print('     x_err:     ', np.array(x_err))
    
print(' \n ')
print(' eField_vals:      ', np.array(eField_vals))
print(' chargeYield_mean: ', np.array(chargeYield_mean))
print(' x_err:            ', np.array(x_err))
print(' chargeYield_std : ', np.array(chargeYield_std))
print(' chargeYield_sem : ', np.array(chargeYield_sem))
'''

# TODO: Those plots are outdated (chargeYield not anymore a function of track_gravicentre_x but as a function of hit_position_x)
'''
x_err = [] # error for the eField values
for eField in range(len(eField_vals)):
    x_err.append(0.05)

out = display(progress(0,len(ly_mean_bin_i_list[eField]),'GC processed'),display_id=True)
for GC in range(len(ly_mean_bin_i_list[eField])):
    lightYield_mean = []
    lightYield_std  = []
    lightYield_sem  = []
    for eField in range(n_field_intensities):
        field_intensity = (eField+1)/10.
        #print(' GC: ', GC, '\t eField: ', eField, '\t ly: ', ly_mean_bin_i_list[eField][GC], ' \t ly sem: ', ly_sem_bin_i_list[eField][GC])
        if not (np.isnan(ly_mean_bin_i_list[eField][GC]) or np.isnan(ly_sem_bin_i_list[eField][GC]) or ly_sem_bin_i_list[eField][GC]==0.):
            lightYield_mean.append(ly_mean_bin_i_list[eField][GC])
            lightYield_std.append(ly_std_bin_i_list[eField][GC])
            lightYield_sem.append(ly_sem_bin_i_list[eField][GC])
        else:
            lightYield_mean.append(float('nan'))
            lightYield_std.append(float('nan'))
            lightYield_sem.append(float('nan'))

    series_mean       = [lightYield_mean,np.array(chargeYield_mean)/10.]
    series_err        = [np.nanmean(lightYield_sem),np.array(chargeYield_sem)/10.]
    #print(' ly mean: ', series_mean[0])
    #print(' ly err:  ',series_err[0])
    #print(' cy mean: ',series_mean[1])
    #print(' cy err:  ',series_err[1])
    
    series_label_list = ['LCM Light Yield [p.e.]','Pixel Charge Yield [ke] (scale factor: 1/10)']
    axis_labels = ['E Field Intensity [kV/cm]','Yield']
    x_min,x_max = -0.05,1.05
    y_min,y_max = 0,1500
    save_name = 'plots/TGraphError_LightAndChargeYield_vs_eField_GC_bin_'+ str(GC) + '.png'
    plot_TGraphErr(eField_vals,series_mean,x_err,series_err,series_label_list,axis_labels,x_min,x_max,y_min,y_max,save_name)
    out.update(progress(GC+1,len(ly_mean_bin_i_list[eField]),'GCs processed'))
out.update(progress(len(ly_mean_bin_i_list[eField]),len(ly_mean_bin_i_list[eField]),'GCs processed'))
'''
print('Done.')

Done.


#### Event Display

In [43]:
# Define which TChain to be used
chain = 15

input_tree = input_tree_list[chain]
n_tracks = input_tree.GetEntries()

print(' n_tracks:          ', input_tree.GetEntries())

plot_folder = str(eField_intensity_list[chain]) + '_kV_per_cm'
print(' plot_folder: ', plot_folder)

# Delete old eventDisplays
os.system('rm -rf plots/' + str(plot_folder) +'/eventDisplays')
os.system('mkdir plots/' + str(plot_folder) +'/eventDisplays')

for track in range(n_tracks):
    input_tree.GetEntry(track)
    
    if track>10:
        break
    
    if track%200==0:
        print(' Processing track', track, 'of', n_tracks, 'in total...')

    # Apply Selection Cuts (those should be the same as at the very beginning!)
    # Note: Make a selected_trackID list does not work since there are >1 files in a TChain.
    # Could, e.g., use unique unix_ts or so.
    # For the moment, just apply exactly the same selection cuts as above.
    if input_tree_list[chain].t_track_length < cut_lenght:
        continue
    if input_tree_list[chain].t_event_ntracks > cut_ntracks:
        continue
    if abs(max(input_tree_list[chain].t_track_start_pos_y,input_tree_list[chain].t_track_end_pos_y)) < cut_throughgoing or\
       abs(min(input_tree_list[chain].t_track_start_pos_y,input_tree_list[chain].t_track_end_pos_y)) < cut_throughgoing:
        #print(input_tree_list[chain].t_track_start_pos_y)
        #print(input_tree_list[chain].t_track_end_pos_y)
        #print(' ---------------- ')
        continue
    
    print(' --------------- ')
    print(' trackID: ', input_tree.t_trackID)
    print(' start x: ', input_tree.t_track_start_pos_x)
    print(' end x:   ', input_tree.t_track_end_pos_x)
    print(' phi:     ', input_tree_list[chain].t_track_phi, ' \t / \t ', input_tree_list[chain].t_track_phi*180./np.pi)
    print(' theta:   ', input_tree_list[chain].t_track_theta, ' \t / \t ', input_tree_list[chain].t_track_theta*180./np.pi)

    
    hits_x = []
    hits_y = []
    hits_z = []
    hits_q = []

    '''    
    print(' ----------------------------------------- ')
    #print(' t_eventID:        ', input_tree.t_eventID)
    print(' t_trackID:        ', input_tree.t_trackID)
    #print(' t_event_q:        ', input_tree.t_event_q)
    #print(' t_track_q:        ', input_tree.t_track_q)
    #print(' t_event_nhits:    ', input_tree.t_event_nhits)
    print(' t_track_start:    [%.2f , %.2f , %.2f]' %(input_tree.t_track_start_pos_x,input_tree.t_track_start_pos_y,input_tree.t_track_start_pos_z))
    print(' t_track_end:      [%.2f , %.2f , %.2f]' %(input_tree.t_track_end_pos_x,input_tree.t_track_end_pos_y,input_tree.t_track_end_pos_z))
    #print(' max(start_y):     ', max(input_tree.t_track_start_pos_y,input_tree.t_track_end_pos_y))
    #print(' min(start_y):     ', min(input_tree.t_track_start_pos_y,input_tree.t_track_end_pos_y))
    print(' t_track_nhits:    ', input_tree.t_track_nhits)
    #print(' t_track_res_x:    ', input_tree.t_track_residual_x)
    #print(' t_track_res_y:    ', input_tree.t_track_residual_y)
    #print(' t_track_res_z:    ', input_tree.t_track_residual_z)
    #print(' sqrt(res_sum2):   ', np.sqrt(input_tree.t_track_residual_x**2 + input_tree.t_track_residual_y**2 + input_tree.t_track_residual_z**2  ))
    
    #print(' l_LCM_tot :      ', input_tree.l_tlcm1_tot)
    '''
    
    # Loop over all hits
    for hit in range(input_tree.t_track_nhits):
        hits_x.append(input_tree.t_track_hits_x[hit])
        hits_y.append(input_tree.t_track_hits_y[hit])
        hits_z.append(input_tree.t_track_hits_z[hit])
        hits_q.append(input_tree.t_track_hits_q[hit])
    
    
    # Plot 2D projection (onto xy pixel tile)
    x_min    = pixels_x_min
    x_max    = pixels_x_max
    y_min    = pixels_y_min
    y_max    = pixels_y_max
    n_bins_x = n_pixels_x
    n_bins_y = n_pixels_y
    x_bins = np.linspace(x_min,x_max,n_bins_x)
    y_bins = np.linspace(y_min,y_max,n_bins_y)
    #save_name = 'plots/'+str(plot_folder)+'/eventDisplays/trackID_'+str(input_tree.t_trackID)+'_2D.png'
    save_name = 'plots/'+str(plot_folder)+'/eventDisplays/chain_'+str(chain)+'_trackID_'+str(input_tree.t_trackID)+'_2D.png'
    plot_event_2D([hits_x,hits_y,hits_q],x_bins,y_bins,['Coordinate x [mm]','Coordinate y [mm]','Charge Deposition [ke]'],save_name)
    
    
    # Plot 3D event display
    x_min    = pixels_x_min
    x_max    = pixels_x_max
    y_min    = pixels_y_min
    y_max    = pixels_y_max
    z_min    = 0
    z_max    = max(pixels_z_max,np.max(hits_z))
    n_bins_x = n_pixels_x
    n_bins_y = n_pixels_y
    n_bins_z = n_pixels_z
    x_bins = np.linspace(x_min,x_max,n_bins_x)
    y_bins = np.linspace(y_min,y_max,n_bins_y)
    z_bins = np.linspace(z_min,z_max,n_bins_z)
    #save_name = 'plots/'+str(plot_folder)+'/eventDisplays/trackID_'+str(input_tree.t_trackID)+'_3D.png'
    save_name = 'plots/'+str(plot_folder)+'/eventDisplays/chain_'+str(chain)+'_trackID_'+str(input_tree.t_trackID)+'_3D.png'
    plot_event_3D([hits_x,hits_y,hits_z,hits_q],x_bins,y_bins,z_bins,\
                  ['Coordinate x [mm]','Coordinate y [mm]','Coordinate z [mm]','Charge Deposition [ke]'],save_name)
    '''
    # OLD VERSION
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim((pixels_x_min,pixels_x_max))
    ax.set_ylim((pixels_y_min,pixels_y_max))
    ax.set_zlim((0.,max(3200,np.max(hits_z))))
    img = ax.scatter(hits_x,hits_y,hits_z,c=hits_q,cmap=plt.hot())
    fig.colorbar(img)
    plt.show()
    save_name = 'evDisplay_3D.png'
    plt.savefig(save_name, dpi=400)
    plt.close()
    '''
    
print('Done.')

 n_tracks:           15905
 plot_folder:  0.4_kV_per_cm
 Processing track 0 of 15905 in total...
 --------------- 
 trackID:  10
 start x:  77.35040283203125
 end x:    51.099483489990234
 phi:      1.6602439880371094  	 / 	  95.1249734764947
 theta:    1.5894619226455688  	 / 	  91.06945986434043
 --------------- 
 trackID:  12
 start x:  126.6305923461914
 end x:    51.308326721191406
 phi:      1.8172787427902222  	 / 	  104.12240216072001
 theta:    1.5216138362884521  	 / 	  87.1820508680385
Done.


### Testing Voxelisation algorithm

In [44]:
'''
# Get all hits in the EVENT (!) and make voxelisation
voxels = np.zeros((n_voxels_x, n_voxels_y, n_voxels_z))

for hit in range(10): #input_tree.t_event_nhits):
    #print(' hit: ', hit, ' \t x: ', input_tree.t_event_hits_x[hit], '\t y: ', input_tree.t_event_hits_y[hit], ' \t z: ', input_tree.t_event_hits_z[hit])

    voxel_x = math.floor((input_tree.t_event_hits_x[hit]+(pitch_x*(n_voxels_x)/2.))/pitch_x)
    voxel_y = math.floor((input_tree.t_event_hits_y[hit]+(pitch_y*(n_voxels_y)/2.))/pitch_y)
    voxel_z = math.floor((input_tree.t_event_hits_z[hit]+(pitch_z*(n_voxels_z)/2.))/pitch_z)

    #print(' voxel_x: ', voxel_x, ' \t voxel_y: ', voxel_y, ' \t voxel_z: ', voxel_z)
    if voxel_x<n_voxels_x and voxel_y<n_voxels_y and voxel_z<n_voxels_z:
        voxels[voxel_x][voxel_y][voxel_z] += input_tree.t_event_hits_q[hit]
    # TODO: make under- and overflow voxel for every coordinate

    h3_event_hits.Fill(input_tree.t_event_hits_x[hit],input_tree.t_event_hits_y[hit],input_tree.t_event_hits_z[hit],input_tree.t_event_hits_q[hit])

for vox_x in range(n_voxels_x):
    vox_x_middle = x_min + (vox_x+0.5)*pitch_x
    for vox_y in range(n_voxels_y):
        vox_y_middle = y_min + (vox_y+0.5)*pitch_y
        for vox_z in range(n_voxels_z):
            vox_z_middle = z_min + (vox_z+0.5)*pitch_z
            if voxels[vox_x][vox_y][vox_z] > 0:
                ntuple.Fill(vox_x_middle,vox_y_middle,vox_z_middle,voxels[vox_x][vox_y][vox_z])
                #h3_event_hits.Fill(vox_x_middle,vox_y_middle,vox_z_middle,voxels[vox_x][vox_y][vox_z])
'''
print('Done.')

Done.
